In [63]:
from urllib.request import urlopen
import certifi
from db.db import *
import pandas as pd
from datetime import datetime
import json
import calendar
from functools import reduce
from operator import mul

In [56]:
db = Database()

In [57]:
API_KEY = "e558vSI8LyRrCGv5TqWJtFTAcMDoVD7c"
HISTORICAL_PRICE_URL = f"https://financialmodelingprep.com/api/v3/historical-price-full/%s?apikey={API_KEY}&from=%s&to=%s"

In [53]:
def get_jsonparsed_data(url):
    response = urlopen(url, cafile=certifi.where())
    data = response.read().decode("utf-8")
    return json.loads(data)

In [51]:
def corps_total_rank_by_standard_columns(df, column_map) -> pd.DataFrame:
    report_df = df.copy()
    rank_df = pd.DataFrame(index=report_df.index)
    total_value = pd.Series([0] * len(report_df), index=report_df.index)

    for col, ascending in column_map.items():
        total_value += report_df[col].rank(ascending=ascending)

    rank_df['total_rank'] = total_value.rank()
    rank_df = rank_df.sort_values(by='total_rank')
    report_df['rank'] = rank_df['total_rank']

    return report_df.loc[rank_df.index]

In [6]:
aapl_financial_report_df = db.fetch_ticker_financial_report(ticker = "AAPL", from_date = "2024-01-01", to_date = "2024-12-01")

In [4]:
earning_report_df = pd.read_csv('earning.csv')

In [7]:
aapl_earning_report_df = earning_report_df[earning_report_df['symbol'] == 'AAPL'][earning_report_df['date'].between("2024-01-01", "2024-12-01")]

/var/folders/d9/3tp2f4s97kn5_bdphqnnqfjw0000gn/T/ipykernel_93489/1841658139.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  aapl_earning_report_df = earning_report_df[earning_report_df['symbol'] == 'AAPL'][earning_report_df['date'].between("2024-01-01", "2024-12-01")]


In [11]:
aapl_financial_report_df = aapl_financial_report_df.drop_duplicates(subset=['date']).sort_values(by='date', ascending=False)

In [12]:
aapl_earning_report_df = aapl_earning_report_df.sort_values(by='date', ascending=False)

In [23]:
aapl_earning_report_df['pegr'] = aapl_financial_report_df['pegr'].to_list()

In [24]:
aapl_earning_report_df

,symbol,date,year,month,day,actual,estimated,earning_rate,earning_growth,earning_growth_prev,earning_growth_prev_prev,earning_rate_prev,earning_rate_prev_prev,earning_rate_prev_prev_prev,pegr
766,AAPL,2024-10-31,2024,10,31,1.64,1.60,1.025000,1.171429,0.915033,0.701835,1.037037,1.020000,1.038095,-1.908962
767,AAPL,2024-08-01,2024,8,1,1.40,1.35,1.037037,0.915033,0.701835,1.493151,1.020000,1.038095,1.050360,-4.417107
768,AAPL,2024-05-02,2024,5,2,1.53,1.50,1.020000,0.701835,1.493151,1.158730,1.038095,1.050360,1.058824,-0.927183
769,AAPL,2024-02-01,2024,2,1,2.18,2.10,1.038095,1.493151,1.158730,0.828947,1.050360,1.058824,1.062937,0.449390


In [22]:
aapl_financial_report_df['pegr'].to_list()

[-1.9089615381647222,
 -4.417107422352988,
 -0.9271831184048985,
 0.4493904021734763]

In [35]:
stocks_df = earning_report_df.copy()

for ticker in stocks_df['symbol'].drop_duplicates():
    current_df = stocks_df[stocks_df['symbol'] == ticker]
    report_df = db.fetch_ticker_financial_report(ticker = "AAPL", from_date = None, to_date = None).drop_duplicates(subset=['date'])
    length = min(len(current_df), len(report_df))
    
    current_df = current_df.iloc[:length].sort_values(by='date', ascending=False)
    report_df = report_df.iloc[:length].sort_values(by='date', ascending=False)
    current_df['pegr'] = report_df['pegr'].to_list()
    
    index = current_df[current_df['symbol'] == ticker].index
    
    prev_df = current_df.shift(-1)
    prev_prev_df = current_df.shift(-2)
    prev_prev_prev_df = current_df.shift(-3)
    
    current_actual = current_df['actual']
    prev_actual = prev_df['actual']
    prev_prev_actual = prev_prev_df['actual']
    prev_prev_prev_actual = prev_prev_prev_df['actual']
    
    stocks_df.loc[index, 'pegr'] = current_df['pegr']
    stocks_df.loc[index, 'earning_growth'] = 1 + (current_actual - prev_actual) / prev_actual
    stocks_df.loc[index, 'earning_growth_prev'] = 1 + (prev_actual - prev_prev_actual) / prev_prev_actual
    stocks_df.loc[index, 'earning_growth_prev_prev'] = 1 + (prev_prev_actual - prev_prev_prev_actual) / prev_prev_prev_actual
    
    stocks_df.loc[index, 'earning_rate_prev'] = 1 + (prev_actual - prev_df['estimated']) / prev_df['estimated']
    stocks_df.loc[index, 'earning_rate_prev_prev'] = 1 + (prev_prev_actual - prev_prev_df['estimated']) / prev_prev_df['estimated']
    stocks_df.loc[index, 'earning_rate_prev_prev_prev'] = 1 + (prev_prev_prev_actual - prev_prev_prev_df['estimated']) / prev_prev_prev_df['estimated']

In [49]:
stocks_df['date'] = pd.to_datetime(stocks_df['date'])
stocks_df

,symbol,date,year,month,day,actual,estimated,earning_rate,earning_growth,earning_growth_prev,earning_growth_prev_prev,earning_rate_prev,earning_rate_prev_prev,earning_rate_prev_prev_prev,pegr
0,A,2024-11-25,2024,11,25,1.46000,1.41000,1.035461,1.106061,1.081967,0.945736,1.047619,1.025210,1.057377,-1.908962
1,A,2024-08-21,2024,8,21,1.32000,1.26000,1.047619,1.081967,0.945736,0.934783,1.025210,1.057377,1.029851,-4.417107
2,A,2024-05-29,2024,5,29,1.22000,1.19000,1.025210,0.945736,0.934783,0.965035,1.057377,1.029851,1.043796,-0.927183
3,A,2024-02-27,2024,2,27,1.29000,1.22000,1.057377,0.934783,0.965035,1.125984,1.029851,1.043796,1.007937,0.449390
4,A,2023-11-20,2023,11,20,1.38000,1.34000,1.029851,0.965035,1.125984,0.927007,1.043796,1.007937,1.053846,1.846951
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282419,ZYXI,2013-05-08,2013,5,8,-0.01000,-0.00333,3.003003,-1.000000,1.000000,0.500000,0.500000,0.500000,1.000000,0.117852
282420,ZYXI,2013-03-14,2013,3,14,0.01000,0.02000,0.500000,1.000000,0.500000,-6.134969,0.500000,1.000000,2.202703,3.421349
282421,ZYXI,2012-11-07,2012,11,7,0.01000,0.02000,0.500000,0.500000,-6.134969,NaN,1.000000,2.202703,NaN,-0.836161
282422,ZYXI,2011-08-03,2011,8,3,0.02000,0.02000,1.000000,-6.134969,NaN,NaN,2.202703,NaN,NaN,-0.562763


In [70]:
years = range(2020, 2025)
months = (3, 6, 9, 12)

now = datetime.now()

total_earn_rates = []

for year in years:
    for month in months:
        if year == now.year and month >= now.month: break
            
        stock_start_date = pd.to_datetime(f"{year}-{month}-1") - pd.DateOffset(months=3)
        stock_end_date = pd.to_datetime(f"{year}-{month}-1") - pd.Timedelta(1, unit='d')
        
        quarter_stocks_df = stocks_df[
            stocks_df['date'].between(stock_start_date, stock_end_date)
        ].drop_duplicates(subset='symbol')
        earning_stocks_df = quarter_stocks_df.query(
#             'pegr < 0.5 and '
            'earning_rate > 1.1 and ' + 
            'earning_rate_prev > 1 and ' +
            'earning_rate_prev_prev > 1 and ' +
#             'earning_rate_prev_prev_prev > 1 and ' +
            'earning_growth > 1.1 and ' +
            'earning_growth_prev > 1.1 and '+
#             'earning_growth_prev_prev > 1 and '+
            'actual > 0'
        )
        selected_stocks_df = corps_total_rank_by_standard_columns(earning_stocks_df, {
            'actual': False,
#             'earning_rate': False,
            'earning_growth' : False,
#             'earning_rate_prev': False,
#             'earning_growth_prev': False,
#             'earning_rate_prev_prev' : False
        }).head(6)
        
        print(year, f"{month} - {month + 2}", f"from {len(earning_stocks_df)} / {len(quarter_stocks_df)}")
        
        earn_rates = []
        
        price_from_date = f"{year}-{str(month).zfill(2)}-01"
        price_to_date = pd.to_datetime(price_from_date) + pd.DateOffset(months=3) - pd.Timedelta(1, unit='d')
        price_to_date = price_to_date.strftime('%Y-%m-%d')
        
        for ticker in selected_stocks_df['symbol']:
            try:
                historical_price = pd.DataFrame(
                    get_jsonparsed_data(HISTORICAL_PRICE_URL % (ticker, price_from_date, price_to_date))['historical']
                )
                from_price = historical_price.iloc[-1]['adjClose']
                to_price = historical_price.iloc[0]['adjClose']
                
                earn_rate = 1 + (to_price - from_price) / from_price
                earn_rates.append(earn_rate)

                print(ticker, from_price, to_price, earn_rate)
            except Exception as e:
                print(ticker, e)
                None
        
        earn_rate = sum(earn_rates, 0.0) / len(earn_rates)
        total_earn_rates.append(earn_rate)
        print(earn_rate)

2020 3 - 5 from 66 / 2832


/var/folders/d9/3tp2f4s97kn5_bdphqnnqfjw0000gn/T/ipykernel_93489/654040892.py:2: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


PIPR 64.54 51.61 0.7996591261233343
SYNA 69.709999 63.720001 0.9140726138871413
MTH 63.84 68.04 1.0657894736842106
AMSF 46.97 42.34 0.9014264424100491
CRTO 12.34 10.25 0.8306320907617504
LEN 58.8 56.93 0.9681972789115647
0.9132961709630085
2020 6 - 8 from 23 / 3689


/var/folders/d9/3tp2f4s97kn5_bdphqnnqfjw0000gn/T/ipykernel_93489/654040892.py:2: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


PFSI 32.36 50.02 1.5457354758961683
SFM 25.48 23.35 0.9164050235478807
VRTX 286.640015 279.119995 0.9737649329944391
GHC 341.62 409.4 1.1984075873777882
GLDD 9.32 9.37 1.0053648068669527
DG 181.47 191.68 1.0562627431531384
1.115990094972728
2020 9 - 11 from 57 / 3759


...

DECK 150.55 182.32 1.2110262371305214
PIPR 187.29 210.86 1.125847615996583
HCI 98.28 95.14 0.968050468050468
HII 287.65 250.23 0.8699113505996872
BBWI 44.92 51.08 1.13713268032057
SPLK 156.16 156.9 1.0047387295081969
1.0527845136010043
2024 6 - 8 from 38 / 4198


/var/folders/d9/3tp2f4s97kn5_bdphqnnqfjw0000gn/T/ipykernel_93489/654040892.py:2: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


ZBRA 307.63 345.38 1.1227123492507234
PGR 209.2 252.1 1.2050669216061185
MEDP 389.41 355.27 0.9123289078349297
THC 133.69 165.84 1.2404817114219464
SWAV 'historical'
RLI 139.53 150.43 1.0781194008456962
1.1117418581918828
2024 9 - 11 from 60 / 4077


/var/folders/d9/3tp2f4s97kn5_bdphqnnqfjw0000gn/T/ipykernel_93489/654040892.py:2: DeprecationWarning: cafile, capath and cadefault are deprecated, use a custom context instead.
  response = urlopen(url, cafile=certifi.where())


RCL 159.28 244.06 1.5322702159718733
OCN 'historical'
PPC 45.81 51.61 1.1266099104998908
FTDR 47.69 58.6 1.2287691339903544
POWL 152.48 267.38 1.7535414480587619
IDCC 134.88 195.96 1.452846975088968
1.4188075367219697


In [71]:
reduce(mul, total_earn_rates)

5.3594652090918125